# xmitgcm llcreader demo

This notebooks contains a demonstration of reading and visulizing data from the
[NAS ECCO Data Portal](https://data.nas.nasa.gov/ecco/).
It makes use of the following software libraries:

- [xmitgcm](https://xmitgcm.readthedocs.io/en/latest/llcreader.html): provides the `llcreader` module which makes all of this work
- [xarray](http://xarray.pydata.org/en/latest/): the basic data structures and computational library for the datasets 
- [dask](https://dask.org/): the parallel computing library which enables lazy representations of huge arrays
- [holoviews](http://holoviews.org/): interactive visualizations

We start by importing the necessary libraries

In [ ]:
import xmitgcm.llcreader as llcreader
%matplotlib inline
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

### Initialize Model Object

[xmitgcm.llcreader]() contains pre-configured references to known LLC model configurations.
We create one of these references as follows:

In [ ]:
model = llcreader.ECCOPortalLLC2160Model()
model

### Create Xarray Dataset

The `model` object can generate xarray datasets for us.
In this example, we generate a dataset which contains the SST for the full model integration.
The `type='latlon'` keyword tells xmitgcm to just show us the "Lat Lon" (LL) part of the LLC grid.

In [ ]:
ds_sst = model.get_dataset(varnames=['Theta'], k_levels=[0], type='latlon')
ds_sst

This dataset is "lazy"; it doesn't actually load any data from the server until required for computation or plotting.
That's a good thing, because it represents over 4 TB of data.

In [ ]:
ds_sst.nbytes / 1e12

### Dynamic Interactive Visualization

Here we create an interactive map of SST which automatically resamples the fields at a resolution appropriate for our screen.

In [ ]:
dataset = hv.Dataset(ds_sst.Theta.isel(k=0).astype('f4'))
hv_im = (dataset.to(hv.Image, ['i', 'j'], dynamic=True)
                .options(cmap='Magma', width=950, height=600, colorbar=True))

%output holomap='scrubber' fps=3
regrid(hv_im, precompute=True)